### Get burnt geopoints

In [64]:
import json
import csv
import shapely

# Example of the CSV file content (assuming it's already read as lines)
csv_file_path = '/Users/sonavagarwal/Documents/GitHub/hackmit-24/gather-training-data/BurnedAreaPatch.csv'

# List to hold the geopoints
burnt_geopoints = []

# Open and parse the CSV file
with open(csv_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    
    # Skip the header
    next(reader)
    
    for row in reader:
        if (len(row) == 4):
            # The GeoJSON-like data is in the fourth column (row[3])
            geojson_data = json.loads(row[3])
            
            # turn geojson into shape 
            shape = shapely.geometry.shape(geojson_data)

            # return the centroid of the shape
            centroid = shape.centroid

            # Append the centroid to the list
            lat = centroid.y
            lon = centroid.x
            burnt_geopoints.append((lat, lon))

# Now burnt_geopoints contains all the points from the polygons
# Example: print the first few geopoints
print(burnt_geopoints[:5])


[(39.838037062490486, -121.38485276665034), (39.77515499260212, -121.38709855486064), (39.84252863891108, -121.38709855486064), (39.82456233322869, -121.39383591949156), (39.84252863891108, -121.39159013128126)]


### Get not burnt geopoints

In [65]:
import random

def generate_random_coordinates_in_tracy():
    # Coordinates of the upper left and bottom right points
    upper_left_lat = 37.753769564438734
    upper_left_lon = -121.46080223281191
    bottom_right_lat = 37.72498664651281
    bottom_right_lon = -121.41662178760858
    
    # Generate random latitude and longitude within the bounds
    random_lat = random.uniform(bottom_right_lat, upper_left_lat)
    random_lon = random.uniform(upper_left_lon, bottom_right_lon)
    
    return random_lat, random_lon


In [66]:
import random



burnt_coordinates = 
safe_coordinates = [generate_random_coordinates_in_tracy() for _ in range(len(burnt_geopoints))]


In [67]:
for burnt in burnt_coordinates[:5]:
    print(burnt)
for safe in safe_coordinates[:5]:
    print(safe)

(39.72799344018584, -121.5263374238992)
(39.77964656902271, -121.54205794137127)
(39.65837400566657, -121.64311841083472)
(39.69542951113651, -121.50780967116422)
(39.8021044511257, -121.56002424705368)
(37.75226997100211, -121.44235297081656)
(37.739051752941144, -121.44701352116435)
(37.75133650192345, -121.42463643984414)
(37.72827945334688, -121.45280194266743)
(37.75110588848532, -121.45643771525282)


### Get lists of coordinates of houses in certain areas

In [68]:
import osmnx as ox

# Function to get houses near a specific coordinate within a given distance
def get_houses_near_coordinate(lat, lon, dist=500):
    """
    Get house footprints near a specific coordinate within a given radius.

    Parameters:
        lat (float): Latitude of the center point.
        lon (float): Longitude of the center point.
        dist (int): Radius in meters to search around the point. Defaults to 500 meters.

    Returns:
        GeoDataFrame: A GeoDataFrame containing house footprints near the coordinate.
    """
    # Create a point from the provided coordinates
    point = (lat, lon)
    
    try:
        # Use the correct OSMnx function to get geometries around a point within the specified distance
        gdf = ox.features_from_point(point, tags={'building': True}, dist=dist)
    
        return gdf
    except Exception as e:
        return None

# Function to extract house centroids and their approximate dimensions (width and height)
def extract_house_centroids_and_dimensions(gdf):
    """
    Extract centroids, widths, and heights from a GeoDataFrame of house footprints.

    Parameters:
        gdf (GeoDataFrame): A GeoDataFrame containing house footprints.

    Returns:
        DataFrame: A DataFrame containing the centroids, widths, and heights of the houses.
    """

    # Compute centroids before changing the CRS (to keep in lat/lon)
    gdf['centroid'] = gdf.centroid
    
    # Extract lat/lon from the centroids and ensure correct order (lat, lon)
    gdf['centroid_lat'] = gdf['centroid'].apply(lambda point: point.y)
    gdf['centroid_lon'] = gdf['centroid'].apply(lambda point: point.x)
    
    # Project the GeoDataFrame to UTM (or a local CRS that uses meters for accurate width/height)
    gdf_meters = gdf.to_crs(epsg=3857)  # Web Mercator (meters)
    
    # Compute approximate width and height of each house (bounding box dimensions) in meters
    gdf_meters['width'] = gdf_meters.bounds['maxx'] - gdf_meters.bounds['minx']
    gdf_meters['height'] = gdf_meters.bounds['maxy'] - gdf_meters.bounds['miny']
    
    # Create a DataFrame with centroids (lat/lon) and dimensions in meters
    house_info = gdf_meters[['width', 'height']].copy()
    house_info['centroid_lat'] = gdf['centroid_lat']
    house_info['centroid_lon'] = gdf['centroid_lon']
    
    return house_info

# # Example usage:
# lat, lon = 37.36311012427722, -122.09531602289128
# houses_gdf = get_houses_near_coordinate(lat, lon, dist=20)
# house_info = extract_house_centroids_and_dimensions(houses_gdf)

# # Print the results
# print(house_info)


In [69]:
import pandas as pd

# empty dataframe to store all the burnt houses
all_burnt_houses = []

for burnt in burnt_coordinates[:5]:
    print(burnt)
    lat, lon = burnt
    houses_gdf = get_houses_near_coordinate(lat, lon, dist=20)
    if houses_gdf is None:
        continue
    house_info = extract_house_centroids_and_dimensions(houses_gdf)
    for index, row in house_info.iterrows():
        all_burnt_houses.append((row['centroid_lat'], row['centroid_lon'], row['width'], row['height']))
        print(row['centroid_lat'], row['centroid_lon'], row['width'], row['height'])

# empty dataframe to store all the safe houses
all_safe_houses = []
for safe in safe_coordinates[:5]:
    lat, lon = safe
    houses_gdf = get_houses_near_coordinate(lat, lon, dist=20)
    if houses_gdf is None:
        continue
    house_info = extract_house_centroids_and_dimensions(houses_gdf)
    for index, row in house_info.iterrows():
        all_safe_houses.append((row['centroid_lat'], row['centroid_lon'], row['width'], row['height']))


(39.72799344018584, -121.5263374238992)


/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


(39.77964656902271, -121.54205794137127)


/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


(39.65837400566657, -121.64311841083472)


/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


(39.69542951113651, -121.50780967116422)


/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


(39.8021044511257, -121.56002424705368)


/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/Users/sonavagar

In [70]:
for burnt in all_burnt_houses[:5]:
    print(burnt)

print("\n\n")
for safe in all_safe_houses[:5]:
    print(safe)




(37.752468496015936, -121.44223779681278, 18.256396487355232, 18.443821829743683)
(37.75247813936651, -121.44241055339364, 17.031882090494037, 24.77949368953705)
(37.75204645854766, -121.4422014807867, 17.922438018023968, 20.27401447482407)
(37.752036317316346, -121.44241256317939, 17.031882090494037, 24.2161783920601)
(37.739123866282874, -121.44686361886693, 18.4790354706347, 24.493487308733165)


### get images from list of coordinates

In [71]:
import pyproj

def calculate_bounding_box(lat, lon, width, height):
    """
    Calculate the bounding box in Web Mercator coordinates (EPSG:3857) 
    given a center latitude and longitude, along with the desired width and height in meters.
    
    Parameters:
    lat (float): Latitude of the center point.
    lon (float): Longitude of the center point.
    width (float): Width of the bounding box in meters.
    height (float): Height of the bounding box in meters.

    Returns:
    tuple: Bounding box as (xmin, ymin, xmax, ymax) in meters.
    """
    # Define the Web Mercator (EPSG:3857) projection
    web_mercator = pyproj.Proj(init='epsg:3857')
    
    # Convert the center point to Web Mercator (meters)
    x_center, y_center = web_mercator(lon, lat)
    
    # Calculate the half width and half height to determine the bounding box
    half_width = width / 2
    half_height = height / 2
    
    # Calculate the bounding box in meters
    x_min = x_center - half_width
    x_max = x_center + half_width
    y_min = y_center - half_height
    y_max = y_center + half_height
    
    return (x_min, y_min, x_max, y_max)



import urllib.parse

def construct_naip_url_encoded(bbox):
    """
    Construct the NAIP image export URL using the bounding box coordinates, with URL encoding.

    Parameters:
    bbox (tuple): Bounding box as (xmin, ymin, xmax, ymax) in meters.

    Returns:
    str: The formatted and URL-encoded NAIP exportImage URL with the bounding box.
    """
    base_url = "https://map.dfg.ca.gov/arcgis/rest/services/Base_Remote_Sensing/NAIP_2016/ImageServer/exportImage"
    bbox_str = f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}"
    
    # URL encode the bounding box string
    bbox_encoded = urllib.parse.quote(bbox_str)
    
    # Construct the full URL with the bounding box
    url = f"{base_url}?bbox={bbox_encoded}&bboxSR=&size=&imageSR=&time=&format=jpgpng&pixelType=U8&noData=&noDataInterpretation=esriNoDataMatchAny&interpolation=+RSP_BilinearInterpolation&compression=&compressionQuality=&bandIds=&mosaicRule=&renderingRule=&f=image"
    
    return url

